***GENERATED CODE FOR churnpredict PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def renameColumns(df, functionsData, applyOn):
    for functionData in functionsData:
        df = df.rename(
            columns={functionData['oldName']: functionData['newName']})
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '181', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '181', 'mean': '', 'stddev': '', 'min': '327-8732', 'max': '422-8344', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone'}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '181', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intl_Plan'}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '181', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VMail_Plan'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '181', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'vmailers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'cluster_labels'}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run churnpredictHooks.ipynb
try:
	#sourcePreExecutionHook()

	predictivechurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/PredictiveChurnTrain.csv', 'filename': 'PredictiveChurnTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run churnpredictHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(predictivechurntrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/PredictiveChurnTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/PredictiveChurnTrain.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/61388f036d4d6b149231665a1631097099/0part.csv", "requestRatio": 7.0, "totalRows": 3332, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 25, "numberOfRows": 3332, "duplicateRowCount": 0, "stats": [{"column": "State", "alias": "State", "generated": 0, "type": "String", "max": "WY", "min": "AK", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Account_Length", "alias": "Account_Length", "generated": 0, "type": "numeric", "max": 243, "min": 1, "mean": 101.05672268907563, "missing": 0.0, "stddev": 39.83}, {"column": "Area_Code", "alias": "Area_Code", "generated": 0, "type": "numeric", "max": 510, "min": 408, "mean": 437.1890756302521, "missing": 0.0, "stddev": 42.38}, {"column": "Phone", "alias": "Phone", "generated": 0, "type": "String", "max": "422-9964", "min": "327-1058", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Intl_Plan", "alias": "Intl_Plan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "VMail_Plan", "alias": "VMail_Plan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "VMail_Message", "alias": "VMail_Message", "generated": 0, "type": "numeric", "max": 51, "min": 0, "mean": 8.093937575030012, "missing": 0.0, "stddev": 13.69}, {"column": "Day_Mins", "alias": "Day_Mins", "generated": 0, "type": "real", "max": 350.8, "min": 0.0, "mean": 179.74948979591838, "missing": 0.0, "stddev": 54.46}, {"column": "Day_Calls", "alias": "Day_Calls", "generated": 0, "type": "numeric", "max": 165, "min": 0, "mean": 100.4327731092437, "missing": 0.0, "stddev": 20.07}, {"column": "Day_Charge", "alias": "Day_Charge", "generated": 0, "type": "real", "max": 59.64, "min": 0.0, "mean": 30.55795318127251, "missing": 0.0, "stddev": 9.26}, {"column": "Eve_Mins", "alias": "Eve_Mins", "generated": 0, "type": "real", "max": 363.7, "min": 0.0, "mean": 200.98142256902761, "missing": 0.0, "stddev": 50.72}, {"column": "Eve_Calls", "alias": "Eve_Calls", "generated": 0, "type": "numeric", "max": 170, "min": 0, "mean": 100.11464585834334, "missing": 0.0, "stddev": 19.93}, {"column": "Eve_Charge", "alias": "Eve_Charge", "generated": 0, "type": "real", "max": 30.91, "min": 0.0, "mean": 17.083631452581034, "missing": 0.0, "stddev": 4.31}, {"column": "Night_Mins", "alias": "Night_Mins", "generated": 0, "type": "real", "max": 395.0, "min": 23.2, "mean": 200.85888355342135, "missing": 0.0, "stddev": 50.58}, {"column": "Night_Calls", "alias": "Night_Calls", "generated": 0, "type": "numeric", "max": 175, "min": 33, "mean": 100.11044417767107, "missing": 0.0, "stddev": 19.57}, {"column": "Night_Charge", "alias": "Night_Charge", "generated": 0, "type": "real", "max": 17.77, "min": 1.04, "mean": 9.038733493397359, "missing": 0.0, "stddev": 2.28}, {"column": "Intl_Mins", "alias": "Intl_Mins", "generated": 0, "type": "real", "max": 20.0, "min": 0.0, "mean": 10.23736494597839, "missing": 0.0, "stddev": 2.79}, {"column": "total_Mins", "alias": "total_Mins", "generated": 0, "type": "real", "max": 885.0, "min": 284.3, "mean": 591.8271608643457, "missing": 0.0, "stddev": 89.94}, {"column": "Intl_Calls", "alias": "Intl_Calls", "generated": 0, "type": "numeric", "max": 20, "min": 0, "mean": 4.479891956782713, "missing": 0.0, "stddev": 2.46}, {"column": "Intl_Charge", "alias": "Intl_Charge", "generated": 0, "type": "real", "max": 5.4, "min": 0.0, "mean": 2.764600840336135, "missing": 0.0, "stddev": 0.75}, {"column": "Total_Charge", "alias": "Total_Charge", "generated": 0, "type": "real", "max": 96.15, "min": 22.93, "mean": 59.44491896758704, "missing": 0.0, "stddev": 10.5}, {"column": "CustServ_Calls", "alias": "CustServ_Calls", "generated": 0, "type": "numeric", "max": 9, "min": 0, "mean": 1.5630252100840336, "missing": 0.0, "stddev": 1.32}, {"column": "Churn", "alias": "Churn", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.14495798319327732, "missing": 0.0, "stddev": 0.35}, {"column": "cluster_labels", "alias": "cluster_labels", "generated": 0, "type": "String", "max": "vmailers", "min": "day_callers", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "test col", "alias": "test col", "generated": 1, "type": "real", "max": 363.7, "min": 0.0, "mean": 200.98142256902761, "missing": 0.0, "stddev": 50.72}]}, "predictionPowerScore": [{"Account_Length": 1.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 1.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 1.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 1.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 1.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0011483426, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0116739729, "CustServ_Calls": 0.0008610997, "Day_Calls": 0.0, "Day_Charge": 1.0, "Day_Mins": 0.9985977145, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0007145042, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.367993509, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.3374360402, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0116716512, "CustServ_Calls": 0.0008594878, "Day_Calls": 0.0, "Day_Charge": 0.998580442, "Day_Mins": 1.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.000714073, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.3679859198, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.3374323406, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 1.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0008036626, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0039831588, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 1.0, "Eve_Mins": 0.998360181, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.000364526, "cluster_labels": 0.0959276146, "test col": 0.998360181, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0039886302, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.9980674817, "Eve_Mins": 1.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0003692773, "cluster_labels": 0.0959266015, "test col": 0.9983177278, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 1.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0012350316, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0179293209, "Intl_Charge": 1.0, "Intl_Mins": 0.9992927612, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.1824482463, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0011379605, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0178253138, "Intl_Charge": 0.9992993447, "Intl_Mins": 1.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.1824023524, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 5.484e-07, "Churn": 5.484e-07, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 5.484e-07, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 1.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 5.484e-07, "Total_Charge": 0.0, "VMail_Message": 5.484e-07, "VMail_Plan": 5.484e-07, "cluster_labels": 5.484e-07, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 1.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0002641815, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 8.62717e-05, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 1.0, "Night_Mins": 0.998235729, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.1989630553, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 8.2204e-05, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.9973044922, "Night_Mins": 1.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.1989211428, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 1.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.0032266875, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.004320012, "Day_Mins": 0.0045056754, "Eve_Calls": 0.0, "Eve_Charge": 0.0046010675, "Eve_Mins": 0.0031771176, "Intl_Calls": 0.0, "Intl_Charge": 0.0039843862, "Intl_Mins": 0.003975914, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0033401497, "Night_Mins": 0.004360881, "Phone": 0.0, "State": 1.0, "Total_Charge": 0.0105530122, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.0, "test col": 0.0031771176, "total_Mins": 0.0020162983}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0114875154, "CustServ_Calls": 0.0015688805, "Day_Calls": 0.0, "Day_Charge": 0.4003528279, "Day_Mins": 0.402687588, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.000559988, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 1.0, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.4363424691, "test col": 0.0, "total_Mins": 0.3891479923}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 1.0, "VMail_Plan": 0.7933795212, "cluster_labels": 0.2915726724, "test col": 0.0, "total_Mins": 0.0}, {"Account_Length": 0.019790453, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0017375035, "Day_Charge": 0.051373023, "Day_Mins": 0.0465152226, "Eve_Calls": 0.0, "Eve_Charge": 0.0140062115, "Eve_Mins": 0.0311759429, "Intl_Calls": 0.0, "Intl_Charge": 0.0044891476, "Intl_Mins": 0.0048988808, "Intl_Plan": 0.0, "Night_Calls": 0.0092645916, "Night_Charge": 0.02678727, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.9992302499, "VMail_Plan": 1.0, "cluster_labels": 0.6928353765, "test col": 0.0311759429, "total_Mins": 0.0002682938}, {"Account_Length": 0.0025943261, "Area_Code": 0.0, "Churn": 0.0, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.1832764503, "Day_Mins": 0.1861406393, "Eve_Calls": 0.0, "Eve_Charge": 0.0468690737, "Eve_Mins": 0.0447765284, "Intl_Calls": 0.0, "Intl_Charge": 0.0866503488, "Intl_Mins": 0.0869744522, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0813462635, "Night_Mins": 0.0798837099, "Phone": 0.0, "State": 0.0007012694, "Total_Charge": 0.2579912111, "VMail_Message": 0.0367308541, "VMail_Plan": 0.0292571448, "cluster_labels": 1.0, "test col": 0.0447765284, "total_Mins": 0.2380124349}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0039886302, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.9980674817, "Eve_Mins": 0.9983177278, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.0, "VMail_Message": 0.0, "VMail_Plan": 0.0003692773, "cluster_labels": 0.0959266015, "test col": 1.0, "total_Mins": 0.0}, {"Account_Length": 0.0, "Area_Code": 0.0, "Churn": 0.0100714693, "CustServ_Calls": 0.0, "Day_Calls": 0.0, "Day_Charge": 0.0, "Day_Mins": 0.0, "Eve_Calls": 0.0, "Eve_Charge": 0.0, "Eve_Mins": 0.0, "Intl_Calls": 0.0, "Intl_Charge": 0.0, "Intl_Mins": 0.0, "Intl_Plan": 0.0, "Night_Calls": 0.0, "Night_Charge": 0.0170038192, "Night_Mins": 0.0, "Phone": 0.0, "State": 0.0, "Total_Charge": 0.3755608616, "VMail_Message": 0.0, "VMail_Plan": 0.0, "cluster_labels": 0.4288879043, "test col": 0.0, "total_Mins": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "editname", "applyOn": [{"columnName": "test col", "type": "real", "min": "0.0", "max": "363.7", "mean": "201.0"}], "functionsData": [{"oldName": "Eve_Mins", "newName": "test col", "asNewColumn": 1, "column": {"columnName": "Eve_Mins", "type": "real", "min": "0.0", "max": "363.7", "mean": "201.0"}}]}], "functionChanges": [{"columnName": "test col", "functionName": "Edit Column Name", "Type": "real", "Parameters": [{"oldName": "Eve_Mins", "newName": "test col", "asNewColumn": 1, "column": {"columnName": "Eve_Mins", "type": "real", "min": "0.0", "max": "363.7", "mean": "201.0"}}]}], "fileheader": [{"field": "State", "alias": "State", "generated": 0, "position": 1, "type": "String"}, {"field": "Account_Length", "alias": "Account_Length", "generated": 0, "position": 2, "type": "numeric"}, {"field": "Area_Code", "alias": "Area_Code", "generated": 0, "position": 3, "type": "numeric"}, {"field": "Phone", "alias": "Phone", "generated": 0, "position": 4, "type": "String"}, {"field": "Intl_Plan", "alias": "Intl_Plan", "generated": 0, "position": 5, "type": "String"}, {"field": "VMail_Plan", "alias": "VMail_Plan", "generated": 0, "position": 6, "type": "String"}, {"field": "VMail_Message", "alias": "VMail_Message", "generated": 0, "position": 7, "type": "numeric"}, {"field": "Day_Mins", "alias": "Day_Mins", "generated": 0, "position": 8, "type": "real"}, {"field": "Day_Calls", "alias": "Day_Calls", "generated": 0, "position": 9, "type": "numeric"}, {"field": "Day_Charge", "alias": "Day_Charge", "generated": 0, "position": 10, "type": "real"}, {"field": "Eve_Mins", "alias": "Eve_Mins", "generated": 0, "position": 11, "type": "real"}, {"field": "Eve_Calls", "alias": "Eve_Calls", "generated": 0, "position": 12, "type": "numeric"}, {"field": "Eve_Charge", "alias": "Eve_Charge", "generated": 0, "position": 13, "type": "real"}, {"field": "Night_Mins", "alias": "Night_Mins", "generated": 0, "position": 14, "type": "real"}, {"field": "Night_Calls", "alias": "Night_Calls", "generated": 0, "position": 15, "type": "numeric"}, {"field": "Night_Charge", "alias": "Night_Charge", "generated": 0, "position": 16, "type": "real"}, {"field": "Intl_Mins", "alias": "Intl_Mins", "generated": 0, "position": 17, "type": "real"}, {"field": "total_Mins", "alias": "total_Mins", "generated": 0, "position": 18, "type": "real"}, {"field": "Intl_Calls", "alias": "Intl_Calls", "generated": 0, "position": 19, "type": "numeric"}, {"field": "Intl_Charge", "alias": "Intl_Charge", "generated": 0, "position": 20, "type": "real"}, {"field": "Total_Charge", "alias": "Total_Charge", "generated": 0, "position": 21, "type": "real"}, {"field": "CustServ_Calls", "alias": "CustServ_Calls", "generated": 0, "position": 22, "type": "numeric"}, {"field": "Churn", "alias": "Churn", "generated": 0, "position": 23, "type": "numeric"}, {"field": "cluster_labels", "alias": "cluster_labels", "generated": 0, "position": 24, "type": "String"}, {"field": "test col", "alias": "test col", "generated": 1, "position": 25, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run churnpredictHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "181", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Length", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "104.73", "stddev": "40.02", "min": "3.0", "max": "205.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Account_Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Area_Code", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "438.61", "stddev": "43.1", "min": "408.0", "max": "510.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Area_Code"}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "181", "mean": "", "stddev": "", "min": "327-8732", "max": "422-8344", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "181", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intl_Plan"}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "181", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VMail_Plan"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "VMail_Message", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "7.17", "stddev": "12.92", "min": "0.0", "max": "46.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "VMail_Message"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "174.4", "stddev": "54.72", "min": "19.5", "max": "335.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "99.62", "stddev": "21.59", "min": "45.0", "max": "150.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "29.65", "stddev": "9.3", "min": "3.32", "max": "57.04", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "203.15", "stddev": "54.47", "min": "42.5", "max": "354.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "98.64", "stddev": "20.41", "min": "48.0", "max": "164.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "17.27", "stddev": "4.63", "min": "3.61", "max": "30.11", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "193.69", "stddev": "46.61", "min": "57.5", "max": "317.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "101.52", "stddev": "19.55", "min": "61.0", "max": "155.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "8.72", "stddev": "2.1", "min": "2.59", "max": "14.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "10.3", "stddev": "3.1", "min": "0.0", "max": "18.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "581.54", "stddev": "92.72", "min": "301.5", "max": "842.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "4.45", "stddev": "2.64", "min": "0.0", "max": "13.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "2.78", "stddev": "0.84", "min": "0.0", "max": "4.91", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "58.42", "stddev": "10.83", "min": "25.52", "max": "90.46", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustServ_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "1.62", "stddev": "1.38", "min": "0.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustServ_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "0.1", "stddev": "0.3", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Churn"}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "181", "mean": "", "stddev": "", "min": "day_callers", "max": "vmailers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "cluster_labels"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "test col", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "181", "mean": "203.15", "stddev": "54.47", "min": "42.5", "max": "354.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "test col"}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "181", "mean": "16.96", "stddev": "13.19", "min": "0.0", "max": "49.0", "missing": "0"}, "updatedLabel": "State_stringindexer_trans..."}, {"feature": "Phone_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "181", "mean": "90.0", "stddev": "52.39", "min": "0.0", "max": "180.0", "missing": "0"}, "updatedLabel": "Phone_stringindexer_trans..."}, {"feature": "Intl_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "181", "mean": "0.09", "stddev": "0.28", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Intl_Plan_stringindexer_t..."}, {"feature": "VMail_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "181", "mean": "0.25", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "VMail_Plan_stringindexer_..."}, {"feature": "cluster_labels_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "181", "mean": "2.24", "stddev": "1.68", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "cluster_labels_stringinde..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run churnpredictHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["State_stringindexer", "Account_Length", "Area_Code", "Phone_stringindexer", "Intl_Plan_stringindexer", "VMail_Plan_stringindexer", "VMail_Message", "Day_Mins", "Day_Calls", "Day_Charge", "Eve_Mins", "Eve_Calls", "Eve_Charge", "Night_Mins", "Night_Calls", "Night_Charge", "Intl_Mins", "total_Mins", "Intl_Calls", "Intl_Charge", "Total_Charge", "CustServ_Calls", "cluster_labels_stringindexer", "test col"], "Churn")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

